In [2]:
import pymongo
import pandas as pd
import numpy as np
import re
import time
import datetime
import nltk
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bson.objectid import ObjectId

In [3]:
client = pymongo.MongoClient('localhost',27017)
db = client.db
comments = db.comments
companies = db.companylist
matches = db.matches
chunks = db.chunks

df = pd.DataFrame.from_records(comments.find({'Label':{'$ne': 0}}))

In [3]:
'''nltk.download('vader_lexicon')
sir = SentimentIntensityAnalyzer()
sent = df.loc[77,'Title'] + ' ' + df.loc[77,'Body']
sir.polarity_scores(sent)['compound']
count = 0

for i in range(len(df)):
    sent = df.loc[i,'Title'] + ' ' + df.loc[i,'Body']
    score = sir.polarity_scores(sent)['compound']
    if (score < 0 and df.loc[i, 'Label'] == 'bullish') or (score > 0 and df.loc[i,'Label'] == 'bearish'):
        print(str(score) + ':' + df.loc[i, 'Label'])
        count = count + 1'''

"nltk.download('vader_lexicon')\nsir = SentimentIntensityAnalyzer()\nsent = df.loc[77,'Title'] + ' ' + df.loc[77,'Body']\nsir.polarity_scores(sent)['compound']\ncount = 0\n\nfor i in range(len(df)):\n    sent = df.loc[i,'Title'] + ' ' + df.loc[i,'Body']\n    score = sir.polarity_scores(sent)['compound']\n    if (score < 0 and df.loc[i, 'Label'] == 'bullish') or (score > 0 and df.loc[i,'Label'] == 'bearish'):\n        print(str(score) + ':' + df.loc[i, 'Label'])\n        count = count + 1"

In [4]:
import flair
from flair.models import TextClassifier
from flair.data import Sentence

In [5]:
labelDf = pd.DataFrame(columns = ['Label', 'Text'])
labelDf['Text'] = df['Body']
labelDf['Label'] = df['Label']

In [6]:
labelDf['Label'] = '__label__' + labelDf['Label'].astype(str)

In [7]:
labelDf = labelDf.sample(frac = 1)

In [8]:
labelDf.iloc[0: int(len(labelDf)*0.8)].to_csv('data/train.csv', sep = '\t', index = False, header = False)
labelDf.iloc[int(len(labelDf)*0.8): int(len(labelDf)*0.9)].to_csv('data/test.csv', sep = '\t', index = False, header = False)
labelDf.iloc[int(len(labelDf)*0.9): ].to_csv('data/dev.csv', sep = '\t', index = False, header = False)

In [9]:
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
corpus = ClassificationCorpus(Path('data/'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train('model/', max_epochs=10)

): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): CrossEntropyLoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-11-01 20:32:32,804 ----------------------------------------------------------------------------------------------------
2020-11-01 20:32:32,805 Corpus: "Corpus: 1246 train + 156 dev + 156 test sentences"
2020-11-01 20:32:32,809 ----------------------------------------------------------------------------------------------------
2020-11-01 20:32:32,811 Parameters:
2020-11-01 20:32:32,814  - learning_rate: "0.1"

2020-11-01 20:32:32,817  - mini_batch_size: "32"
2020-11-01 20:32:32,819  - patience: "3"
2

{'test_score': 0.4551,
 'dev_score_history': [0.4423,
  0.3782,
  0.4423,
  0.4615,
  0.5128,
  0.4679,
  0.4167,
  0.5449,
  0.4744,
  0.4936],
 'train_loss_history': [1.0977133634762886,
  1.0779983324882312,
  1.0570394029984107,
  1.0267436580780225,
  1.0338823245121882,
  1.032352504057762,
  1.0101591654312916,
  1.0075173484973419,
  0.9957530926435422,
  0.9986650133744265],
 'dev_loss_history': [1.0662189722061157,
  1.0878649950027466,
  1.075381875038147,
  1.0330644845962524,
  1.043900728225708,
  1.058959722518921,
  1.0392714738845825,
  0.9996782541275024,
  1.0420639514923096,
  1.0387905836105347]}

In [4]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('model/best-model.pt')

2020-11-02 18:11:40,431 loading file model/best-model.pt


In [20]:
sentence = Sentence('stonks are flat')
classifier.predict(sentence)
sentence.labels[0].to_dict()

{'value': 'neutral', 'confidence': 0.46689432859420776}

In [5]:
df = pd.DataFrame.from_records(db.threads.find({'Label':{'$ne': 0}}))

In [12]:
count = 0
for i in df['Body']:
    print(count)
    doc = i
    strings = doc.split('. ')
    for sent in strings:
        sentence = Sentence(sent)
        classifier.predict(sentence)
        print(sentence.labels)
    count = count + 1

0
[bullish (0.4264)]
[bearish (0.4014)]
[bullish (0.3883)]
[neutral (0.361)]
[neutral (0.5081)]
[neutral (0.4587)]
[neutral (0.5477)]
[neutral (0.6959)]
[neutral (0.4369)]
1
[neutral (0.4986)]
[bullish (0.4714)]
[neutral (0.3696)]
[bullish (0.469)]
[bearish (0.4374)]
[bullish (0.583)]
[neutral (0.5378)]
[neutral (0.4533)]
[bullish (0.3903)]
[neutral (0.4696)]
[neutral (0.5345)]
[bullish (0.3929)]
[bearish (0.4591)]
[bearish (0.4687)]
[bullish (0.6079)]
[bullish (0.5661)]
[bullish (0.4594)]
[neutral (0.4239)]
[bullish (0.3992)]
[bullish (0.3883)]
[bullish (0.3863)]
[neutral (0.4566)]
[bearish (0.4039)]
[neutral (0.4199)]
[neutral (0.6094)]
[neutral (0.6532)]
[neutral (0.3901)]
2
[neutral (0.4543)]
[neutral (0.4557)]
[bullish (0.485)]
[bearish (0.4268)]
[bullish (0.4372)]
[neutral (0.3578)]
[neutral (0.6233)]
[bearish (0.4631)]
[neutral (0.3811)]
[neutral (0.5655)]
[neutral (0.5073)]
[neutral (0.3623)]
[neutral (0.5085)]
[neutral (0.5124)]
[neutral (0.6283)]
[neutral (0.4525)]
3
[bullish

KeyboardInterrupt: 

In [13]:
sentence.labels

[]